# Dashboard

Cuaderno que presenta el dashboard creado para el concurso. Utiliza los conjuntos de datos y modelos generados en los cuadernos anteriores. Cuenta con tres pestañas que se presentan en el pop-up inicial.

In [2]:
from dash import Dash, html, dcc, Input, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import json
import requests
from dash import dash_table
import dash_bootstrap_components as dbc
import re
import dash_daq as daq
import pickle
from io import BytesIO
import dash
import warnings
warnings.filterwarnings('ignore')

#Carga de los conjuntos de datos preprocesados
matricula = pd.read_csv("smart/matricula.csv")
medias = pd.read_csv("smart/medias.csv")
notas = pd.read_csv("smart/notas.csv")
top5 = pd.read_csv("smart/top5.csv")
matriculasG2 = pd.read_csv("smart/matriculaG2.csv")

colegios = pd.read_csv("smart/colegiosHasta22.csv")
poblacionJoven = pd.read_csv("smart/poblacionJoven22.csv")
trainSimulador = pd.read_csv("smart/train_simulador.csv")
predecirSimulador = pd.read_csv("smart/predecir_simulador.csv")

#Variables globales
cursos_matricula = ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']
cursos_acceso = ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
matricula_marks = {
    i: {'label': course, 'style': {'color': 'black',"fontFamily": "Arial, sans-serif"}}
    for i, course in enumerate(cursos_matricula)
}
acceso_marks = {
    i: {'label': course, 'style': {'color': 'black',"fontFamily": "Arial, sans-serif"}}
    for i, course in enumerate(cursos_acceso)
}
centro_mapa = dict(lat=40.4168, lon=-3.7038)
grupos_densidad = ["Muy poco densa", "Poco densa", "Moderadamente densa", "Densa", "Muy densa"]


#Mapa pestaña 1
def mapa_p1(universidad_seleccionada, curso_seleccionado, densidad_seleccionada, valor_zoom, centro_mapa):
    
    #Aplicamos los filtros seleccionados
    matricula_filtrada = matricula[(matricula['universidad'] == universidad_seleccionada)&(matricula['curso_academico'].isin(curso_seleccionado))&(matricula['poblacion'].isin(densidad_seleccionada))]
    
    #Añadimos los ajustes de la figura
    fig = px.scatter_mapbox(matricula_filtrada,lat="lat_municipio_residencia",lon="lon_municipio_residencia",color="log_count",size_max=20,opacity=0.8,color_continuous_scale=px.colors.sequential.Bluered,hover_name="municipio",hover_data={"count": True})
    global_min_log = np.log(1)
    global_max_log = np.log(26000)
    tick_vals = np.linspace(global_min_log, global_max_log, num=10)
    tick_texts = [str(int(round(np.exp(v)))) for v in tick_vals]
    fig.update_layout(
        coloraxis_colorbar=dict(title="Estudiantes",tickvals=tick_vals,ticktext=tick_texts,tickangle=0),
        coloraxis=dict(cmin=global_min_log,cmax=global_max_log),
        mapbox_style="carto-positron",
        autosize=True,
        showlegend=False,
        mapbox=dict(center=centro_mapa,zoom=valor_zoom),
        margin=dict(l=15, r=0, t=0, b=15)
    )
    points_size = 15
    fig.update_traces(marker={'size': points_size},hovertemplate=("<b>%{hovertext} </b><br>" +"Matriculados: %{customdata[0]}<extra></extra>"))
    
    return fig

#Diagrama de sectores pestaña 1
def sectores_p1(universidad_seleccionada, curso_seleccionado):
    
    #Aplicamos los filtros seleccionados y agrupamos por grupos poblacionales
    matricula_filtrada = matricula[(matricula['universidad'] == universidad_seleccionada)&(matricula['curso_academico'] == curso_seleccionado)]
    matricula_agrupada = matricula_filtrada.groupby('poblacion', as_index=False)['count'].sum()
    
    #Forzamos a que siempre salgan los sectores en el mismo orden y con los mismos colores
    orden_grupos = ['Muy densa', 'Densa', 'Moderadamente densa', 'Poco densa', 'Muy poco densa']
    matricula_agrupada['poblacion'] = pd.Categorical(matricula_agrupada['poblacion'], categories=orden_grupos, ordered=True)
    matricula_agrupada = matricula_agrupada.sort_values('poblacion') 
    color_palette = {'Muy densa': '#1f77b4','Densa': '#ff7f0e','Moderadamente densa': '#2ca02c','Poco densa': '#9467bd','Muy poco densa': '#d62728'  }
    
    #Añadimos los ajustes a la figura
    pie_chart = px.pie(matricula_agrupada,values='count',names='poblacion',title=f"Distribución de población en la {universidad_seleccionada.upper()} curso {curso_seleccionado}",color='poblacion',color_discrete_map=color_palette,category_orders={'poblacion': orden_grupos})
    pie_chart.update_layout(
        autosize=True,
        title={'text': f"Universidad {universidad_seleccionada.upper()}, curso {curso_seleccionado}",'y': 0.3,  'x': 1.0,  'xanchor': 'right', 'yanchor': 'middle','font': {'family': 'Arial, sans-serif','size': 16, 'color': 'black',}},
        margin={'l': 50,'r': 50,'t': 10,'b': 10},
        legend=dict(title=dict(text="Densidad poblacional",font=dict(size=16,family="Arial, sans-serif")),),
    )
    pie_chart.update_traces(hovertemplate=("<b>%{label} </b><br>"+"Matriculados: %{value}<extra></extra>"))
    
    return pie_chart

#Mapa pestaña 2
#Cargamos el fichero que contiene las delimitaciones territoriales. Nos quedamos solo con provincias y ciudades autónomas
geojson_url = "https://raw.githubusercontent.com/ufoe/d3js-geojson/refs/heads/master/Spain.json"
response = requests.get(geojson_url)
geojson_datos = response.json()
provincias = {"type": "FeatureCollection","features": [feature for feature in geojson_datos["features"]
        if feature["properties"].get("type_en") in ["Autonomous Community", "Autonomous City"]]}

def mapa_p2(provincias, customdata=None):
    
    #Añadimos los ajustes a la figura
    figC = px.choropleth_mapbox(geojson=provincias,locations=[feature["properties"]["name"] for feature in provincias["features"]],featureidkey="properties.name",color_discrete_sequence=["#FF6961"],mapbox_style="carto-positron",center={"lat": 40.4168, "lon": -3.7038},zoom=5.0,opacity=0.8)
    if customdata:
        figC.data[0].customdata = customdata
    figC.update_traces(hovertemplate=("<b>%{location}</b><br>"+"Nota media: %{customdata:.2f}<extra></extra>"))
    figC.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0},clickmode="event+select",showlegend=False)

    return figC


#Diagrama caja y bigote pestaña 2
def caja_y_bigote_p2(curso_seleccionado, df):
    
    #Forzamos el orden de las categorías
    orden_grupos = ["Muy poco densa", "Poco densa", "Moderadamente densa", "Densa", "Muy densa"]
    df["poblacion"] = pd.Categorical(df["poblacion"], categories=orden_grupos, ordered=True)
    df = df.sort_values("poblacion")
    
    #Añadimos las cajas a la figura
    fig = go.Figure()
    leyenda_usada = set()  
    for provincia, group in df.groupby("des_provincia_centro_sec"):
        color = "blue" if provincia == "Global" else "red"
        datos_existentes = group.dropna(subset=["percentil_25", "percentil_50", "percentil_75", "nota_minima", "nota_maxima"])
        fig.add_trace(go.Box(
            name=provincia,  
            x=datos_existentes["poblacion"],  
            q1=datos_existentes["percentil_25"], 
            median=datos_existentes["percentil_50"], 
            q3=datos_existentes["percentil_75"], 
            lowerfence=datos_existentes["nota_minima"], 
            upperfence=datos_existentes["nota_maxima"], 
            boxpoints=False,
            marker_color=color,
            legendgroup=provincia,  
            showlegend=provincia not in leyenda_usada)
        )
        leyenda_usada.add(provincia)

    if(provincia!="Global"):
        fig.update_layout(title=f"Curso {curso_seleccionado} en {provincia}")
    else:
        fig.update_layout(title=f"Curso {curso_seleccionado}")
    fig.update_layout(
        yaxis_title="Nota de acceso",
        xaxis=dict(tickmode="array",tickvals=orden_grupos),
        boxmode="group",  
        plot_bgcolor="white",
        yaxis=dict(gridcolor="lightgrey",zerolinecolor="grey",zerolinewidth=1,range=[0, 14]),
        margin=dict(l=40, r=40, t=40, b=40),
    )
    
    return fig

#Tabla pestaña 2
def tabla_p2(top5):
    data=top5.to_dict('records')
    #Propiedades de la tabla
    return dash_table.DataTable(
        columns=[{"name": "Orden", "id": "orden"},{"name": "Titulación", "id": "des_titulacion"},{"name": "Matriculados", "id": "count"}],
        data=top5.to_dict('records'),
        style_table={"overflowX": "auto","margin": "20px","width": "95%","borderLeft": "1px solid #ccc","borderRight": "1px solid #ccc"},
        style_cell={"textAlign": "center","padding": "5px","fontFamily": "Arial, sans-serif"},
        style_header={"fontWeight": "bold","backgroundColor": "#f4f4f4","textAlign": "center"},
    )

#Figura 1 pestaña 3
def figura1_p3(df):
    #Creamos el curso a partir del año
    df['curso_etiqueta'] = df['curso'].astype(int).apply(lambda x: f"{x}-{str((x + 1) % 100).zfill(2)}")
    #Ajustamos el mapeo de colores y las etiquetas a mostrar (1 de cada 3 cursos para no sobrecargar el eje X)
    color_map = {'Muy densa': '#1f77b4','Densa': '#ff7f0e','Moderadamente densa': '#2ca02c','Poco densa': '#9467bd','Muy poco densa': '#d62728'}
    cursos_filtrados = df['curso'][::3].tolist()
    etiquetas_filtradas = df['curso_etiqueta'][::3].tolist()
    x_min = df['curso'].min() - 0.4  
    x_max = df['curso'].max() +0.1 

    #Ajustes de la figura
    fig = go.Figure()
    #Dividimos en dos porque las predicciones (a partir de 2022) irán con línea discontinua
    df_antes_2022 = df[df['curso'] < 2022]
    df_desde_2022 = df[df['curso'] >= 2021]

    categorias = ['colesMMP', 'colesMP', 'colesP', 'colesPP', 'colesMPP']
    nombres = ['Muy densa', 'Densa', 'Moderadamente densa', 'Poco densa', 'Muy poco densa']
    for categoria, nombre in zip(categorias, nombres):
        color = color_map[nombre]
        # Línea continua para cursos antes de 2022
        fig.add_trace(go.Scatter(
            x=df_antes_2022['curso'], y=df_antes_2022[categoria], mode='lines',
            name=f"{nombre}", line=dict(color=color, dash='solid'),
            hovertemplate="Curso %{customdata}, %{y} colegios",
            customdata=df_antes_2022['curso_etiqueta'],
            showlegend=True
        ))
        # Línea discontinua para cursos desde 2022
        fig.add_trace(go.Scatter(
            x=df_desde_2022['curso'], y=df_desde_2022[categoria], mode='lines',
            name=f"{nombre}", line=dict(color=color, dash='dot'),
            hovertemplate="<b>Predicción</b> curso %{customdata}, %{y} colegios",
            customdata=df_desde_2022['curso_etiqueta'],
            showlegend=False
        ))
    fig.update_layout(
        title=None,  
        xaxis_title='Curso',
        yaxis_title='Número de colegios',
        legend_title='',
        legend=dict(title=dict(text="Densidad poblacional",font=dict(size=16,family="Arial, sans-serif" )),traceorder='normal',itemsizing='constant',x=1.05,y=0.5,yanchor='middle'),
        template='plotly_white',
        margin=dict(t=30, l=80, r=10, b=50),
        xaxis=dict(tickmode='array',tickvals=cursos_filtrados,ticktext=etiquetas_filtradas,range=[x_min,x_max],tickangle=-45)
    )

    return fig

#Figura 2 pestaña 3
def figura2_p3(df):
    
    #Ajustamos el mapeo de colores y las etiquetas a mostrar (1 de cada 3 cursos para no sobrecargar el eje X)
    color_map = {'Muy densa': '#1f77b4','Densa': '#ff7f0e','Moderadamente densa': '#2ca02c','Poco densa': '#9467bd','Muy poco densa': '#d62728'}
    cursos_filtrados = df['periodo'][::3].tolist()
    x_min = df['periodo'].min() - 0.4  
    x_max = df['periodo'].max() +0.1 
    
    #Ajustes de la figura    
    fig = go.Figure()
    #Dividimos en dos porque las predicciones (a partir de 2022) irán con línea discontinua
    df_antes_2022 = df[df['periodo'] < 2021]
    df_desde_2022 = df[df['periodo'] >= 2020]
    
    nombres = ['Muy densa', 'Densa', 'Moderadamente densa', 'Poco densa', 'Muy poco densa']
    for nombre in nombres:
        color = color_map[nombre]
        # Línea continua para cursos antes de 2022
        fig.add_trace(go.Scatter(
            x=df_antes_2022['periodo'], y=df_antes_2022[nombre], mode='lines',
            name=f"{nombre}", line=dict(color=color, dash='solid'),
            hovertemplate="Año %{customdata}, % personas dentro de población joven: %{y}",
            customdata=df_antes_2022['periodo'],
            showlegend=True
        ))

        # Línea discontinua para cursos desde 2022
        fig.add_trace(go.Scatter(
            x=df_desde_2022['periodo'], y=df_desde_2022[nombre], mode='lines',
            name=f"{nombre}", line=dict(color=color, dash='dot'),
            hovertemplate="<b>Predicción</b> año %{customdata}, % personas dentro de población joven: %{y}",
            customdata=df_desde_2022['periodo'],
            showlegend=False
        ))

    fig.update_layout(
        title=None,  
        xaxis_title='Año',
        yaxis_title='% Medio de población joven',
        legend=dict(title=dict(text="Densidad poblacional",font=dict(size=16,family="Arial, sans-serif")),traceorder='normal',itemsizing='constant',x=1.05,y=0.5,yanchor='middle'),
        template='plotly_white',
        margin=dict(t=30, l=80, r=10, b=50),  
        xaxis=dict(tickmode='array',tickvals=cursos_filtrados,ticktext=cursos_filtrados,range=[x_min,x_max],tickangle=-45)
    )

    return fig
    
#Figura 3 pestaña 3
def figura3_p3(trainSimulador,predecirSimulador,universidad="total",inputs=[0,0,0,0,0]):
    #Aplicamos los filtros seleccionados para el conjunto de datos históricos y para el dataset de entrada del simulador
    if (universidad == "total"):
        #Con dos sirve
        trainSimulador = trainSimulador[(trainSimulador["uva"]==1) & (trainSimulador["ucm"]==1)]
        predecirSimulador = predecirSimulador[(predecirSimulador["uva"]==1) & (predecirSimulador["ucm"]==1)]
    else:
        if (universidad == "uva"):
            trainSimulador = trainSimulador[(trainSimulador["uva"]==1) & (trainSimulador["ucm"]==0)]
            predecirSimulador = predecirSimulador[(predecirSimulador["uva"]==1) & (predecirSimulador["ucm"]==0)]
        else:
            trainSimulador = trainSimulador[(trainSimulador[universidad]==1) & (trainSimulador["uva"]==0)]
            predecirSimulador = predecirSimulador[(predecirSimulador[universidad]==1) & (predecirSimulador["uva"]==0)]
    
    #Cargamos el modelo y las predicciones para la universidad seleccionada y renombramos sus columnas
    prediccionesMatriculados = pd.read_csv("smart/pm_"+universidad+".csv")
    prediccionesMatriculados = prediccionesMatriculados.rename(columns={'Muy_poco_densa': 'Muy poco densa'})
    prediccionesMatriculados = prediccionesMatriculados.rename(columns={'Poco_densa': 'Poco densa'})
    prediccionesMatriculados = prediccionesMatriculados.rename(columns={'Moderadamente_densa': 'Moderadamente densa'})
    prediccionesMatriculados = prediccionesMatriculados.rename(columns={'Muy_densa': 'Muy densa'})
    url = 'smart/modelo_simulador_'+universidad+'.pkl'
    response = requests.get(url)
    if response.status_code == 200:
        with BytesIO(response.content) as file:
            print("Cargado el modelo de "+universidad)
            modelo_poisson_cargado = pickle.load(file)

    #Agrupamos el histórico y las predicciones a futuro
    df = pd.concat([trainSimulador, prediccionesMatriculados], ignore_index=True)
    df.drop(columns=['uva','ucm','uam','uc3m','urjc','total_poblacion','num_colegios','y'])
    columnas_categorias = ["Muy densa", "Densa", "Moderadamente densa", "Poco densa", "Muy poco densa"]
    columnas_modelo = ["Muy_densa", "Densa", "Moderadamente_densa", "Poco_densa", "Muy_poco_densa"]

    #Transformamos el formato para que aparezca toda la información en una misma línea
    for col in columnas_categorias:
        df[col] = df[col] * df["total_matriculados"]
        df_final = df.groupby("curso_academico")[columnas_categorias].sum().astype(int).reset_index()
    df_final['curso_etiqueta'] = df_final['curso_academico'].astype(int).apply(lambda x: f"{x}-{str((x + 1) % 100).zfill(2)}")

    #Si se han modificado los valores de los inputs del simulador
    if inputs!=[0,0,0,0,0]:
        df_future = predecirSimulador.copy()
        inputs = inputs[::-1]
        #Sumamos los colegios abiertos/cerrados
        for i, densidad_columna in enumerate(columnas_modelo): 
            mask = df_future[densidad_columna] == 1
            df_future.loc[mask, 'num_colegios'] += inputs[i]
        #Realizamos las predicciones encadenadas
        for year in range(2022, 2029):
            df_year = df_future[df_future['curso_academico'] == year].copy()
            df_year['y'] = modelo_poisson_cargado.predict(df_year).round().astype(int)
            df_future.loc[df_future['curso_academico'] == year, 'y'] = df_year['y']
            if year < 2028:
                for _, row in df_year.iterrows():
                    mask = ((df_future['curso_academico'] == year + 1)&(df_future['Muy_densa'] == row['Muy_densa'])&(df_future['Densa'] == row['Densa'])&(df_future['Moderadamente_densa'] == row['Moderadamente_densa'])&(df_future['Poco_densa'] == row['Poco_densa'])&(df_future['Muy_poco_densa'] == row['Muy_poco_densa'])&(df_future['uva'] == row['uva'])&(df_future['ucm'] == row['ucm'])&(df_future['uam'] == row['uam'])&(df_future['uc3m'] == row['uc3m'])&(df_future['urjc'] == row['urjc']))
                    df_future.loc[mask, 'total_matriculados'] = row['y']
                    
        #Renombramos las columnas para quitar los espacios y transformamos igual que antes
        df_future = df_future.rename(columns={'Muy_poco_densa': 'Muy poco densa'})
        df_future = df_future.rename(columns={'Poco_densa': 'Poco densa'})
        df_future = df_future.rename(columns={'Moderadamente_densa': 'Moderadamente densa'})
        df_future = df_future.rename(columns={'Muy_densa': 'Muy densa'})
        df_future = df_future.drop(columns=['uva', 'ucm','uam','uc3m','urjc','total_poblacion','num_colegios','y'])
        for col in columnas_categorias:
            df_future[col] = df_future[col] * df_future["total_matriculados"]
            df_simulado = df_future.groupby("curso_academico")[columnas_categorias].sum().astype(int).reset_index()
        df_simulado['curso_etiqueta'] = df_simulado['curso_academico'].astype(int).apply(lambda x: f"{x}-{str((x + 1) % 100).zfill(2)}")

    #Ajustamos los colores y los parámetros de la figura
    color_map = {'Muy densa': '#1f77b4','Densa': '#ff7f0e','Moderadamente densa': '#2ca02c','Poco densa': '#9467bd','Muy poco densa': '#d62728'}
    
    fig = go.Figure()
    for col in df_final.columns[1:-1]:
        #Datos históricos (línea continua)
        fig.add_trace(go.Scatter(
            x=df_final[df_final["curso_academico"] <= 2022]["curso_academico"], 
            y=df_final[df_final["curso_academico"] <= 2022][col],
            mode='lines',
            name=col.replace("_", " "),
            line=dict(color=color_map[col],dash='solid'),
            hovertemplate="Curso %{customdata}, %{y} matriculados",
            customdata=df_final[df_final["curso_academico"] <= 2022]['curso_etiqueta'],
        ))
        #Predicciones (línea discontinua)
        fig.add_trace(go.Scatter(
            x=df_final[(df_final["curso_academico"] > 2021)&(df_final["curso_academico"] < 2028)]["curso_academico"],
            y=df_final[(df_final["curso_academico"] > 2021)&(df_final["curso_academico"] < 2028)][col],
            mode='lines',
            name=col.replace("_", " "),
            line=dict(color=color_map[col],dash='dot'),
            hovertemplate="<b>Predicción</b> curso %{customdata}, %{y} matriculados",
            customdata=df_final[df_final["curso_academico"] > 2022]['curso_etiqueta'],
            showlegend=False
        ))
        #Simulaciones (línea continua con círculos)
        if inputs!=[0,0,0,0,0]:
            fig.add_trace(go.Scatter(
                x=df_simulado[(df_simulado["curso_academico"] > 2021)&(df_simulado["curso_academico"] < 2028)]["curso_academico"],
                y=df_simulado[(df_simulado["curso_academico"] > 2021)&(df_simulado["curso_academico"] < 2028)][col],
                mode='lines+markers',
                name=col.replace("_", " "),
                line=dict(color=color_map[col],dash='solid'),
                hovertemplate="<b>Simulación</b> curso %{customdata}, %{y} matriculados",
                customdata=df_simulado[df_simulado["curso_academico"] > 2022]['curso_etiqueta'],
                showlegend=False
            ))
    fig.update_layout(
        title=None,
        xaxis_title='Curso',
        yaxis_title='Estudiantes matriculados',
        yaxis=dict(tickformat='d'),
        legend=dict(title=dict(text="Densidad poblacional",font=dict(size=16,family="Arial, sans-serif")),traceorder='normal',itemsizing='constant',x=1.05,y=0.5,yanchor='middle'),
        template='plotly_white',
        margin=dict(t=30, l=80, r=10, b=50),
        xaxis=dict(tickmode='array', tickvals=df_final['curso_academico'], ticktext=df_final['curso_etiqueta'])
    )

    return fig

#Pop-up inicial
modal = dbc.Modal(
    [dbc.ModalBody(html.Div([html.Img(src="/assets/vaciapp.jpg",style={"width": "100%", "height": "auto", "display": "block", "margin": "0 auto"},),])),
     dbc.ModalFooter(dbc.Button("Cerrar", id="close-modal", className="ms-auto", n_clicks=0)),],
    id="welcome-modal",
    is_open=True,  
    size="lg",  
    centered=True,  
    backdrop=True,  
)

#Creamos la aplicación y definimos el layout
app = Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
#app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

app.layout = html.Div(style={'display': 'flex', 'flexDirection': 'column', 'height': '97vh'}, children=[
    modal,
    dcc.Tabs(id='tabs', value='tab1', style={'height': '10%','font-family': 'Arial, sans-serif','fontWeight': 'bold', 'fontSize': '16px'}, children=[
        dcc.Tab(label='Representación por grupos poblacionales en el sistema universitario', value='tab1',style={'font-family': 'Arial, sans-serif','fontWeight': 'bold', 'fontSize': '16px'}),
        dcc.Tab(label='Acceso al sistema universitario', value='tab2',style={'font-family': 'Arial, sans-serif','fontWeight': 'bold', 'fontSize': '16px'}),
        dcc.Tab(label='Impacto de la apertura/cierre de colegios', value='tab3',style={'font-family': 'Arial, sans-serif','fontWeight': 'bold', 'fontSize': '16px'}),
    ]),
    html.Div(id='content', style={'flex': '1', 'display': 'flex', 'height': '90%'}),
    dcc.Store(id='store-zoom', data={'zoom': 5.0, 'center': dict(lat=40.0168, lon=-3.0038)})
])

#Callback para actualizar las pestañas
@app.callback(
    Output('content', 'children'),
    [Input('tabs', 'value')]
)

def actualizar_contenido(tab):
    #PESTAÑA 1
    if tab == 'tab1':
        return [
            #Mapa p1
            html.Div(style={'flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)', 'padding': '5px', 'width': '90%', 'display': 'flex', 'flexDirection': 'column','height': '93vh'}, children=[
                html.Div(style={'padding': '10px'}, children=[
                    html.H3("Municipio de origen de los estudiantes", style={'margin': '0','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','padding-bottom': '0px','font-weight': 'bold'}),
                    html.P("Selecciona la Universidad y el curso académico deseado en las ventanas situadas a la derecha.", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginBottom':'0px','marginTop':'10px'}),
                    html.Div([
                        html.P("También puedes filtrar el mapa por densidad poblacional. ", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginTop':'10px'}),
                        html.Button(
                            '?',
                            id='btn',
                            style={'backgroundColor': '#007BFF','color': 'white','borderRadius': '50%','width': '25px','height': '25px','border': 'none','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','fontSize': '14px','fontWeight': 'bold','cursor': 'pointer','textAlign': 'center','lineHeight': '15px','marginLeft': '10px','transition': 'background-color 0.3s ease, transform 0.2s ease'}
                        ),          
                        dbc.Tooltip(html.Div([
                            html.P("Muy poco densa 1-5.000 habitantes" ),
                            html.P("Poco densa 5.001-25.000 habitantes"),
                            html.P("Moderadamente densa 25.001-100.000 habitantes"),
                            html.P("Densa 100.001-500.000 habitantes"),
                            html.P("Muy densa +500.001 habitantes", style={'marginBottom': '0px'}),
                        ]),target="btn",style={'whiteSpace': 'normal','backgroundColor': '#333','color': 'white','borderRadius': '5px','fontSize': '14px','padding': '5px 10px','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)'}),
                    ],style={'display': 'flex', 'alignItems': 'center'}),
                    dcc.Checklist(
                        id='density-checkbox',
                        options=[{'label': density, 'value': density} for density in grupos_densidad],
                        value=list(grupos_densidad),
                        labelStyle={'display': 'inline-block', 'font-family': 'Arial, sans-serif','font-size': '1.2rem', 'marginBottom': '10px','marginRight': '20px'},
                        inline=True,
                        style={'padding': '10px'}
                    ),
                    html.P("Municipio de origen de estudiantes por universidad, curso y densidad poblacional",style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','display': 'inline-block'}),
                    html.Button(
                        '?',
                        id='btn5',
                        style={'backgroundColor': '#007BFF','color': 'white','borderRadius': '50%','width': '25px','height': '25px','border': 'none','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','fontSize': '14px','fontWeight': 'bold','cursor': 'pointer','textAlign': 'center','lineHeight': '15px','marginLeft': '10px','transition': 'background-color 0.3s ease, transform 0.2s ease','display': 'inline-block'}
                    ),
                    dbc.Tooltip(
                        html.Div([
                            html.P("El mapa muestra burbujas sobre los municipios de origen de los estudiantes matriculados en la universidad y curso seleccionado. Solo aparecen aquellas que se corresponden con municipios que cumplen los filtros de densidad poblacional. El color de las burbujas indica el número de estudiantes matriculados procedente de ese municipio según la escala situada a la derecha.",style={'text-align':'justify','marginBottom': '0px'}),
                        ]),
                        target="btn5",
                        style={'backgroundColor': '#333','color': 'white','borderRadius': '5px','fontSize': '14px','padding': '5px 10px','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)','textAlign': 'left'}
                    ),
                ]),      
                dcc.Graph(
                    id='map',
                    figure=mapa_p1('uc3m', ['2017-18'], grupos_densidad,5.4,dict(lat=40.0168, lon=-3.0038)),
                    config={'scrollZoom': True},
                    style={'height': '100%', 'width': '100%','flexGrow': 1}
                ),
                dcc.Store(id='store-zoom', data={'zoom': 5.0, 'center': dict(lat=40.0168, lon=-3.0038)}),
            ]),
            html.Div(style={'flex': '1', 'display': 'flex', 'flexDirection': 'column', 'padding': '0px', 'gap': '10px',  'marginLeft': '10px','marginRight': '10px','height': '93vh'}, children=[
                html.Div(
                    style={'flex': '1','display': 'flex','flexDirection': 'column','padding': '10px','gap': '20px','border': '1px solid #ccc','borderRadius': '10px','boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'},
                    children=[
                        html.Div(
                            style={'marginBottom': '0px'},
                            children=[
                                html.H3("Filtros para las figuras", style={'marginBottom':'10px','font-weight': 'bold','margin': '0','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','padding-bottom': '0px'}),
                                html.P("Selecciona la Universidad.", style={'marginTop':'20px','font-family': 'Arial, sans-serif','font-size': '1.2rem'}),
                                dcc.RadioItems(
                                    id='university-radio',
                                    options=[{'label': 'UC3M', 'value': 'uc3m'},{'label': 'UVA', 'value': 'uva'},{'label': 'UAM', 'value': 'uam'},{'label': 'UCM', 'value': 'ucm'},{'label': 'URJC', 'value': 'urjc'}],
                                    value='uc3m',
                                    labelStyle={'display': 'inline-block', 'font-family': 'Arial, sans-serif','font-size': '1.0rem', 'marginBottom': '0px','marginRight': '50px'},
                                    style={'padding': '0px'}
                                ),]
                        ),
                        html.Div(
                            children=[
                                html.P("Selecciona el curso académico.", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginTop':'0px'}),
                                html.Div(
                                    children=[
                                        dcc.Slider(
                                            id='course-slider',
                                            min=0,
                                            max=len(cursos_matricula) - 1,
                                            step=1,
                                            marks=matricula_marks,
                                            included=False,
                                            value=0
                                        )
                                    ],
                                    style={'width': '95%', 'margin': '0','white-space': 'nowrap', 'padding':'0px','marginBottom':'10px'}  # Contenedor con el 80% del ancho de la ventana
                                ),],style={'marginBottom': '0px'})
                        ]),  
                #Figura pestaña 1
                html.Div(style={'flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}, children=[
                    html.Div([
                        html.H3("Ratio de estudiantes matriculados por total de habitantes", style={'font-weight': 'bold','margin': '0','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','padding-bottom': '0px','marginBottom': '0px','display': 'inline-flex','alignItems': 'center'}),
                        html.Button(
                            '?',
                            id='btn6',
                            style={'backgroundColor': '#007BFF','color': 'white','borderRadius': '50%','width': '25px','height': '25px','border': 'none','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','fontSize': '14px','fontWeight': 'bold','cursor': 'pointer','textAlign': 'center','lineHeight': '15px','marginLeft': '10px','transition': 'background-color 0.3s ease, transform 0.2s ease','display': 'inline-block'}
                        ),
                        dbc.Tooltip(
                            html.Div([
                                html.P("Para cada grupo poblacional se calcula:" ,style={'marginBottom': '0px','text-align':'justify'}), 
                                html.P("- Por un lado, el total de habitantes censados en los municipios del grupo, que se han matriculado en la universidad seleccionada para el curso correspondiente." ,style={'marginBottom': '0px','text-align':'justify'}),
                                html.P("- Por otro lado, el total de habitantes censados en los municipios del grupo." ,style={'marginBottom': '0px','text-align':'justify'}),
                                html.P("La figura muestra el ratio entre esos dos valores." ,style={'marginBottom': '0px','text-align':'justify','marginTop':'10px'}),
                                ]),
                            target="btn6",
                            style={
                                'backgroundColor': '#333',
                                'color': 'white',
                                'borderRadius': '5px',
                                'fontSize': '14px',
                                'padding': '5px 10px',
                                'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)',
                                'textAlign': 'left',
                            }
                        )
                    ]),
                    dcc.Loading(
                         id="loading-mapa",
                         type="circle",
                         children=[dcc.Graph(id="line-graph",style={'height': '28vh','marginTop':'0px'})])
                ]),
                #Diagrama de sectores pestaña 1
                html.Div(style={'height': '33vh', 'overflow': 'hidden' , 'flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}, children=[
                    html.Div([  
                        html.H3("Distribución de estudiantes por universidad y curso", style={'font-weight': 'bold','margin': '0','marginBottom':'0px','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','padding-bottom': '10px','display': 'inline-flex',}),
                        html.Button(
                            '?',
                            id='btn7',
                            style={'backgroundColor': '#007BFF','color': 'white','borderRadius': '50%','width': '25px','height': '25px','border': 'none','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','fontSize': '14px','fontWeight': 'bold','cursor': 'pointer','textAlign': 'center','lineHeight': '15px','marginLeft': '10px','transition': 'background-color 0.3s ease, transform 0.2s ease','display': 'inline-block'}
                        ),
                        dbc.Tooltip(
                            html.Div([
                                html.P("La figura muestra, para el curso y la universidad seleccionada, la proporción de estudiantes matriculados según su grupo poblacional.",style={'marginBottom': '0px','text-align':'justify'}),
                            ]),
                            target="btn7",
                            style={'backgroundColor': '#333','color': 'white','borderRadius': '5px','fontSize': '14px','padding': '5px 10px','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)','textAlign': 'left'}
                        )
                    ]),
                    dcc.Loading(
                         id="loading-pie",
                         type="circle",
                         children=[
                             dcc.Graph(
                                id='pie-chart',
                                style={'height':'22vh','display': 'flex','justify-content': 'flex-start','align-items': 'flex-start', 'marginTop':'0px','marginBottom':'0px'}
                            )
                         ]),
                ])
               
            ])
        ]
    elif tab == 'tab2':
            return [
            #Mapa pestaña 2
            html.Div(style={'flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)', 'padding': '5px', 'width': '90%', 'display': 'flex', 'flexDirection': 'column','height': '93vh'}, children=[
                html.Div(style={'padding': '10px'}, children=[
                    html.H3("Nota media de acceso a la universidad por provincias y ciudades autónomas", style={'font-weight': 'bold','margin': '0','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','padding-bottom': '0px'}),
                    html.P("Selecciona el curso académico.", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginTop':'20px'}),   
                    html.Div(
                        children=[
                            dcc.Slider(
                                id='course-slider2',
                                min=0,
                                max=len(cursos_acceso) - 1,
                                step=1,
                                marks=acceso_marks,
                                included=False,
                                value=0
                            )],style={'width': '95%', 'margin': '0 auto','white-space': 'nowrap','marginTop':'30px'}  # Contenedor con el 80% del ancho de la ventana
                    ) 
                ]),
                html.Div([
                    html.Div(id="comunidad-seleccionada", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginTop':'10px','marginBottom':'20px','marginLeft':'10px'}),
                ]),
                dcc.Loading(
                         id="loading-mapa",
                         type="circle",
                         children=[
                             dcc.Graph(id="mapa", config={'scrollZoom': True}, 
                                       style={'marginTop':'10px','marginBottom':'10px','marginLeft':'10px','marginRigth':'10px','flexGrow': 1,'height': '65vh','widht':'50%'}),#,'height': '100%', 'width': '100%',}),
                         ])
            ]),
           html.Div(style={'flex': '1', 'display': 'flex', 'flexDirection': 'column', 'padding': '0px', 'gap': '10px',  'marginLeft': '10px','marginRight': '10px','height': '93vh'}, children=[             
                html.Div(style={'flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}, children=[
                    html.Div(style={'display': 'flex', 'alignItems': 'center', 'gap': '10px', 'marginBottom': '20px'}, children=[
                        html.H3("Nota de acceso: Medidas estadísticas por provincias y grupos poblacionales",style={'margin': '0','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','font-weight':'bold'}),
                        html.Button(
                            '?',
                            id='btn2',
                            style={'backgroundColor': '#007BFF','color': 'white','borderRadius': '50%','width': '25px','height': '25px','border': 'none','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','fontSize': '14px','fontWeight': 'bold','cursor': 'pointer','textAlign': 'center','lineHeight': '15px','marginLeft': '0px','transition': 'background-color 0.3s ease, transform 0.2s ease',}
                        ),
                        dbc.Tooltip(html.Div([
                            html.P("En concreto se representan: ", style={'marginBottom': '0px','marginTop': '0px','textAlign': 'justify'}),
                            html.P("\"max\": nota máxima alcanzada", style={'marginTop': '0px','marginBottom': '0px','textAlign': 'left'}),
                            html.P("\"q3\": nota que representa el tercer cuartil (valor por debajo del cual se encuentra el 75% de los datos ordenados)", style={'marginTop': '0px','marginBottom': '0px','textAlign': 'justify'}),
                            html.P("\"median\": nota situada en la mediana (valor por debajo del cual se encuentra el 50% de los datos ordenados)", style={'marginTop': '0px','marginBottom': '0px','textAlign': 'justify'}),
                            html.P("\"q1\": nota que representa el primer cuartil (valor por debajo del cual se encuentra el 25% de los datos ordenados)", style={'marginTop': '0px','marginBottom': '0px','textAlign': 'justify'}),
                            html.P("\"min\": nota mínima alcanzada", style={'marginTop': '0px','marginBottom': '0px','textAlign': 'justify'}),
                            html.P("En el caso de que, para una provincia, un curso y un grupo poblacional concretos, no se represente la caja correspondiente,", style={'marginTop': '10px','marginBottom': '0px','textAlign': 'justify'}),
                            html.P("significa que no hay datos asociados a esa combinación", style={'marginTop': '0px','marginBottom': '0px','textAlign': 'justify'})
                        ]), target="btn2", style={'backgroundColor': '#333','color': 'white','borderRadius': '5px','fontSize': '14px','padding': '5px 10px','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)'}),
                    ]),
                    #Caja y bigote pestaña 2
                    dcc.Loading(
                        id="loading-mapa",
                        type="circle",
                        children=[
                            dcc.Graph(
                                id='caja-bigote',
                                style={'display': 'flex','justify-content': 'flex-start','align-items': 'flex-start','height':'43vh'}
                            ),
                        ]
                    ),
                ]),
               #Tabla pestaña 2
                html.Div(style={'flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}, children=[
                    html.H3("Top 5 titulaciones por curso y grupo poblacional",style={'margin': '0','textAlign': 'left','color': '#000','fontFamily': 'Arial, sans-serif','fontSize': '1.2rem','font-weight': 'bold'}),
                    html.Div([
                        html.P("Selecciona el filtro por densidad poblacional.", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginTop':'10px'}),
                        html.Button(
                            '?',
                            id='btn3',
                            style={'backgroundColor': '#007BFF','color': 'white','borderRadius': '50%','width': '25px','height': '25px','border': 'none','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','fontSize': '14px','fontWeight': 'bold','cursor': 'pointer','textAlign': 'center','lineHeight': '15px','marginLeft': '10px','transition': 'background-color 0.3s ease, transform 0.2s ease'}
                        ),
                        dbc.Tooltip(
                            html.Div([
                                html.P("Muy poco densa 1-5.000 habitantes"),
                                html.P("Poco densa 5.001-25.000 habitantes"),
                                html.P("Moderadamente densa 25.001-100.000 habitantes"),
                                html.P("Densa 100.001-500.000 habitantes"),
                                html.P("Muy densa +500.001 habitantes", style={'marginBottom': '0px'}),
                                html.P("La tabla muestra las cinco titulaciones con más admisiones para el curso y grupo poblacional seleccionados.", style={'marginBottom': '0px','marginTop':'10px','text-align':'justify'}),
                            ]),
                            target="btn3",
                            style={'backgroundColor': '#333','color': 'white','borderRadius': '5px','fontSize': '14px','padding': '5px 10px','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)'}
                        )], style={'display': 'flex','alignItems': 'center'}),
                        dcc.RadioItems(id='poblacion-radio',
                                    options=[
                                        {'label': 'Muy poco densa', 'value': 'Muy poco densa'},
                                        {'label': 'Poco densa', 'value': 'Poco densa'},
                                        {'label': 'Moderadamente densa', 'value': 'Moderadamente densa'},
                                        {'label': 'Densa', 'value': 'Densa'},
                                        {'label': 'Muy densa', 'value': 'Muy densa'}
                                    ],
                                    value='Muy poco densa',
                                    labelStyle={'display': 'inline-block', 'font-family': 'Arial, sans-serif','font-size': '1.0rem', 'marginBottom': '0px','marginRight': '30px'},
                        style={'padding': '10px'}),
                        html.Div(id="top-5-table-container",style={'marginBottom':'0px','marginTop':'0px'}),
                ])
            ])
        ]
    elif tab == 'tab3':
            return [
            #Figura 1 pestaña 3
            html.Div(style={'marginLeft':'10px','flex': '1', 'display': 'flex', 'flexDirection': 'column', 'padding': '0px', 'gap': '10px',  'marginLeft': '0px','marginRight': '0px','height': '93vh'}, children=[             
                html.Div(style={'marginLeft':'10px','flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}, children=[
                    html.H3("Número de colegios por curso y grupo poblacional", style={'margin': '0','font-weight': 'bold','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','padding-bottom': '10px'}),
                    html.P("En linea continua aparecen representados los datos históricos, mientras que la línea ", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginBottom':'0px','marginLeft':'10px','display': 'inline-block'}),
                    html.P("discontinua presenta predicciones futuras basadas en la tendencia histórica.", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginBottom':'0px','marginLeft':'10px','display': 'inline-block','marginTop':'0px'}),
                    html.Button(
                        '?',
                        id='btn4',
                        style={'backgroundColor': '#007BFF','color': 'white','borderRadius': '50%','width': '25px','height': '25px','border': 'none','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','fontSize': '14px','fontWeight': 'bold','cursor': 'pointer','textAlign': 'center','lineHeight': '15px','marginLeft': '10px','transition': 'background-color 0.3s ease, transform 0.2s ease','display': 'inline-block'}
                    ),
                    dbc.Tooltip(
                        html.Div([
                            html.P("Las predicciones a futuro se han realizado con un modelo de regresión lineal basado en el número de estudiantes por grupo poblacional en cursos anteriores.", style={'text-align':'justify'}),#, style={'marginLeft': '65px', 'margin': '0'}),
                            html.P("Muy poco densa 1-5.000 habitantes"),
                            html.P("Poco densa 5.001-25.000 habitantes"),
                            html.P("Moderadamente densa 25.001-100.000 habitantes"),
                            html.P("Densa 100.001-500.000 habitantes"),
                            html.P("Muy densa +500.001 habitantes", style={'marginBottom': '0px'}),
                        ]),
                        target="btn4",
                        style={'backgroundColor': '#333','color': 'white','borderRadius': '5px','fontSize': '14px','padding': '5px 10px','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)','textAlign': 'left'}
                    ),
                    dcc.Loading(
                             id="loading-mapa",
                             type="circle",
                             children=[dcc.Graph(figure=figura1_p3(colegios),style={'marginTop':'10px','marginBottom':'10px','marginLeft':'10px','marginRigth':'10px','flexGrow': 1,'height': '30vh'})])
                    ]),
                #Figura 2 pestaña 3
                html.Div(style={'marginLeft':'10px','flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}, children=[
                    html.H3("Población joven media por grupo poblacional", style={'margin': '0','font-weight': 'bold','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','padding-bottom': '10px'}),
                    html.P("En linea continua aparecen representados los datos históricos, mientras que la línea ", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginBottom':'0px','marginLeft':'10px','display': 'inline-block'}),
                    html.P("discontinua presenta predicciones futuras basadas en la tendencia histórica.", style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginBottom':'0px','marginLeft':'10px','display': 'inline-block','marginTop':'0px'}),
                    html.Button(
                        '?',
                        id='btn10',
                        style={'backgroundColor': '#007BFF','color': 'white','borderRadius': '50%','width': '25px','height': '25px','border': 'none','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','fontSize': '14px','fontWeight': 'bold','cursor': 'pointer','textAlign': 'center','lineHeight': '15px','marginLeft': '10px','transition': 'background-color 0.3s ease, transform 0.2s ease','display': 'inline-block'}
                    ),
                    dbc.Tooltip(
                        html.Div([
                            html.P("Las predicciones a futuro se han realizado con un modelo ARIMA basado en el número medio de personas en población joven en años anteriores.", style={'text-align':'justify'}),#, style={'marginLeft': '65px', 'margin': '0'}),
                            html.P("Muy poco densa 1-5.000 habitantes"),#, style={'marginLeft': '65px', 'margin': '0'}),
                            html.P("Poco densa 5.001-25.000 habitantes"),#, style={'marginLeft': '70px', 'margin': '0'}),
                            html.P("Moderadamente densa 25.001-100.000 habitantes"),#, style={'margin': '0'}),
                            html.P("Densa 100.001-500.000 habitantes"),#, style={'marginLeft': '85px', 'margin': '0'}),
                            html.P("Muy densa +500.001 habitantes", style={'marginBottom': '0px'}),#, style={'marginLeft': '103px', 'margin': '0'})
                        ]),
                        target="btn10",
                        style={'backgroundColor': '#333','color': 'white','borderRadius': '5px','fontSize': '14px','padding': '5px 10px','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)','textAlign': 'left'}
                    ),
                    dcc.Loading(
                             id="loading-mapa",
                             type="circle",
                             children=[dcc.Graph(figure=figura2_p3(poblacionJoven),style={'marginTop':'10px','marginBottom':'10px','marginLeft':'10px','marginRigth':'10px','flexGrow': 1,'height': '30vh'})])
                    ])
            ]),
            #Figura 3 pestaña 3
            html.Div(style={'flex': '1', 'display': 'flex', 'flexDirection': 'column', 'padding': '0px', 'gap': '10px',  'marginLeft': '10px','marginRight': '10px','height': '93vh'}, children=[             
                html.Div(style={'flex': '1', 'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}, children=[
                    html.H3("Simulador para valorar el impacto de la apertura o cierre de colegios en la universidad", style={'margin': '0','font-weight': 'bold','textAlign': 'left','color': '#000','font-family': 'Arial, sans-serif','font-size': '1.2rem','padding-bottom': '20px'}),
                    html.P("A continuación se presenta una herramienta que permite simular el impacto causado por la apertura o cierre de colegios en municipios pertenecientes a diferentes grupos de densidad poblacional en la posterior matriculación en el sistema universitario.", style={'marginTop':'10px','font-family': 'Arial, sans-serif','font-size': '1.2rem'}),
                    html.P("En primer lugar, selecciona una universidad o el total de todas ellas.", style={'marginTop':'10px','font-family': 'Arial, sans-serif','font-size': '1.2rem'}),
                    dcc.RadioItems(
                            id='university-radio5',
                            options=[{'label': 'Total', 'value': 'total'},
                                    {'label': 'UC3M', 'value': 'uc3m'},
                                    {'label': 'UVA', 'value': 'uva'},
                                    {'label': 'UAM', 'value': 'uam'},
                                    {'label': 'UCM', 'value': 'ucm'},
                                    {'label': 'URJC', 'value': 'urjc'}
                            ],
                            value='total',
                            labelStyle={'display': 'inline-block', 'font-family': 'Arial, sans-serif','font-size': '1.0rem', 'marginBottom': '0px','marginRight': '50px'},
                            style={'padding': '0px'}
                    ),
                    html.P("A continuación, para cada uno de los grupos poblacionales, selecciona el numero de colegios que deseas abrir (número positivo) o cerrar (número negativo) y pulsa el botón de 'Simular'", style={'marginTop':'10px','font-family': 'Arial, sans-serif','font-size': '1.2rem'}),
                     html.Div([
                        html.Div([
                            daq.NumericInput(
                                id=f"input-{grupos_densidad[i]}", 
                                min=-100000, 
                                max=100000,
                                value=0,
                                size=70,
                                label=f"{grupos_densidad[i].capitalize()}",  
                                labelPosition="top",
                                style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginBottom': '10px','marginRight': '40px'}
                            ) for i in range(len(grupos_densidad))
                        ], style={'display': 'flex', 'alignItems': 'center'}), 
                        html.Button(
                            "Simular",
                            id="submit-button",
                            n_clicks=0,
                            style={'backgroundColor': '#007BFF','color': '#FFFFFF','border': 'none','padding': '5px 10px','fontSize': '16px','borderRadius': '5px','cursor': 'pointer','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)','transition': 'background-color 0.3s ease, transform 0.2s ease'}
                        ),
                    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginTop': '20px'}),
                    html.P("En línea continua aparecen representados los datos históricos, la línea discontinua presenta predicciones futuras basadas en la tendencia histórica, y la línea continua con círculos ",style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginBottom': '0px','marginLeft': '0px','marginTop':'10px','display': 'inline','verticalAlign': 'middle'}),
                    html.P("presenta las simulaciones realizadas.",style={'font-family': 'Arial, sans-serif','font-size': '1.2rem','marginBottom': '0px','marginLeft': '10px','display': 'inline','verticalAlign': 'middle'}),
                    html.Button(
                        '?',
                        id='btn11',
                        style={'backgroundColor': '#007BFF',  'color': 'white',  'borderRadius': '50%',  'width': '25px',  'height': '25px',  'border': 'none',  'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)',  'fontSize': '14px',  'fontWeight': 'bold',  'cursor': 'pointer',  'textAlign': 'center',  'lineHeight': '25px',  'marginLeft': '10px','transition': 'background-color 0.3s ease, transform 0.2s ease','display': 'inline-block','verticalAlign': 'middle'}),
                    dbc.Tooltip(
                        html.Div([
                            html.P("Las predicciones y simulaciones a futuro se han realizado con un modelo lineal generalizado de Poisson basado en el número medio de personas en población joven, el número de colegios abiertos y el número de estudiantes matriculados en las diferentes universidades en años anteriores.", style={'text-align':'justify'}),#, style={'marginLeft': '65px', 'margin': '0'}),
                            html.P("Muy poco densa 1-5.000 habitantes"),#, style={'marginLeft': '65px', 'margin': '0'}),
                            html.P("Poco densa 5.001-25.000 habitantes"),#, style={'marginLeft': '70px', 'margin': '0'}),
                            html.P("Moderadamente densa 25.001-100.000 habitantes"),#, style={'margin': '0'}),
                            html.P("Densa 100.001-500.000 habitantes"),#, style={'marginLeft': '85px', 'margin': '0'}),
                            html.P("Muy densa +500.001 habitantes", style={'marginBottom': '0px'}),#, style={'marginLeft': '103px', 'margin': '0'})
                        ]),
                        target="btn11",
                        style={'backgroundColor': '#333','color': 'white','borderRadius': '5px','fontSize': '14px','padding': '5px 10px','boxShadow': '0 4px 6px rgba(0, 0, 0, 0.2)','textAlign': 'justify'}
                    ),
                    dcc.Loading(
                             id="loading-mapa",
                             type="circle",
                             children=[dcc.Graph(
                             id="graphBoton",
                             figure=figura3_p3(trainSimulador.copy(),predecirSimulador.copy()),
                             style={'marginTop': '10px', 'marginBottom': '10px', 'marginLeft': '10px', 'marginRight': '10px', 'flexGrow': 1, 'height': '35vh'}
                    )]), 
                ]),
            ])
        ]
          
        
#Zoom de la pestaña 1
@app.callback(
    Output('store-zoom', 'data'),
    [Input('map', 'relayoutData')],
    [State('store-zoom', 'data')]
)
def actualizar_zoom(relayout_data, current_data):
    if relayout_data and 'mapbox.zoom' in relayout_data:
        current_data['zoom'] = relayout_data['mapbox.zoom']
    if relayout_data and 'mapbox.center' in relayout_data:
        current_data['center'] = relayout_data['mapbox.center']
    return current_data

#Mapa de la pestaña 1
@app.callback(
    Output('map', 'figure'),
    [Input('university-radio', 'value'),
     Input('course-slider', 'value'),
     Input('density-checkbox', 'value')],
    [State('store-zoom', 'data')]
)
def actualizar_mapa_p1(universidad_seleccionada, indice_curso_seleccionado, densidad_seleccionada, zoom_data):
    valor_zoom = zoom_data['zoom']
    centro_mapa = zoom_data['center']
    curso_seleccionado = [cursos_matricula[indice_curso_seleccionado]]
    return mapa_p1(universidad_seleccionada, curso_seleccionado, densidad_seleccionada, valor_zoom, centro_mapa)

#Gráfico de sectores de la pestaña 1
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('university-radio', 'value'),
     Input('course-slider', 'value')]
)
def actualizar_sectores_p1(universidad_seleccionada, indice_curso_seleccionado):
    curso_seleccionado = cursos_matricula[indice_curso_seleccionado]
    return sectores_p1(universidad_seleccionada, curso_seleccionado)

#Mapa de la pestaña 2
@app.callback(
    Output("mapa", "figure"),
    Output("comunidad-seleccionada", "children"),
    Input("mapa", "clickData"),
    Input('course-slider2', 'value'),
)
def actualizar_mapa_p2(click_data,indice_curso_seleccionado):
    comunidad_seleccionada = click_data["points"][0]["location"] if click_data else None
    curso_seleccionado = [cursos_acceso[indice_curso_seleccionado]]
    df_filtrado = medias[medias["curso_academico"] == curso_seleccionado[0]]  
    medias_por_provincia = df_filtrado.set_index("des_provincia_centro_sec")["nota_media"].to_dict()
    customdata = [
        medias_por_provincia.get(feature["properties"]["name"], None)
        for feature in provincias["features"]
    ]
    figC = mapa_p2(provincias, customdata)
    mensaje = (
        f"Ha seleccionado {comunidad_seleccionada}" if comunidad_seleccionada 
        else "Selecciona una provincia o ciudad autónoma"
    )
    return figC, mensaje

#Gráfico caja y bigote pestaña 2
@app.callback(
    Output('caja-bigote', 'figure'),
    [Input("mapa", "clickData"),
        Input('course-slider2', 'value')]
)
def actualizar_caja_bigote(click_data,indice_curso_seleccionado):
    comunidad_seleccionada = click_data["points"][0]["location"] if click_data else None
    curso_seleccionado = cursos_acceso[indice_curso_seleccionado]
    notas_filtrado = notas[notas["curso_academico"] == curso_seleccionado]  # Filtrar por el curso
    if (click_data!= None):
        notas_filtrado = notas_filtrado[(notas_filtrado["des_provincia_centro_sec"] == comunidad_seleccionada) | (notas_filtrado["des_provincia_centro_sec"] == "Global")]  # Filtrar por el curso
    else:
        notas_filtrado = notas_filtrado[notas_filtrado["des_provincia_centro_sec"] == "Global"]
    return caja_y_bigote_p2(curso_seleccionado,notas_filtrado)

#Tabla de la pestaña 2
@app.callback(
    Output("top-5-table-container", "children"),
    [Input('poblacion-radio', 'value'),
     Input('course-slider2', 'value')]
)
def actualizar_tabla(poblacion,course_index):
    curso = cursos_acceso[course_index]
    top5_filtrado = top5[top5["poblacion"]==poblacion]
    top5_filtrado = top5_filtrado[top5_filtrado["curso_academico"]==curso]
    mensaje = f"Curso académico seleccionado: {curso}"
    return html.Div([
        html.P(mensaje, style={
            'font-family': 'Arial, sans-serif','font-size': '1.0rem', 'marginBottom':'0', 'marginLeft':'15px'
        }),
        tabla_p2(top5_filtrado)
    ])

#Gráfico de líneas de la pestaña 1
@app.callback(
    Output("line-graph", "figure"),
    Input('university-radio', 'value')
)
def update_graph(universidad):
    max_value = matriculasG2["porcentaje_todos"].max()
    filtered_df = matriculasG2[matriculasG2["universidad"] == universidad].copy()  
    filtered_df['curso_numero'] = filtered_df['curso_academico'].str[:4].astype(int)
    filtered_df['hover_text'] = ("Curso: " + filtered_df['curso_academico'] +", estudiantes " + (filtered_df['porcentaje_todos'] * 100).round(3).astype(str) +"%")
    color_map = {'Muy densa': '#1f77b4','Densa': '#ff7f0e','Moderadamente densa': '#2ca02c','Poco densa': '#9467bd','Muy poco densa': '#d62728'}
    orden_grupos = ['Muy densa', 'Densa', 'Moderadamente densa', 'Poco densa', 'Muy poco densa']
    x_min = filtered_df['curso_numero'].min() - 0.4 
    x_max = filtered_df['curso_numero'].max() +0.1 

    fig = px.line(
        filtered_df, 
        x="curso_numero", 
        y="porcentaje_todos", 
        color="poblacion", 
        labels={"porcentaje_todos": "Estudiantes", "curso_numero": "Año"},
        line_shape="linear",
        color_discrete_map=color_map,
        category_orders={'poblacion': orden_grupos},
        markers=True,
        hover_data={'hover_text': True},
    )
    fig.update_layout(
        xaxis=dict(tickvals=filtered_df['curso_numero'].unique(),  ticktext=filtered_df['curso_academico'].unique(), title=None,gridcolor="lightgrey",automargin=True,range=[x_min,x_max]),
        yaxis=dict(gridcolor="lightgrey",tickformat=".2%"),
        plot_bgcolor="white",
        margin=dict(l=40, r=40, t=40, b=40),
        font=dict(family="Arial, sans-serif",size=14,),
        title={'text': f"Universidad {universidad.upper()}",'y': 0.3,'x': 0.94,'xanchor': 'right','yanchor': 'middle','font': {'family': 'Arial, sans-serif','size': 16,'color': 'black'}},
        legend=dict(title=dict(text="Densidad poblacional",font=dict(size=16,family="Arial, sans-serif")),),
    )
    fig.update_traces(hovertemplate='%{customdata[0]}') 
    return fig

#Simulador pestaña 3
@app.callback(
    Output("graphBoton", "figure"),
    [Input("submit-button", "n_clicks"),
     Input('university-radio5', 'value')],
    [State(f"input-{grupos_densidad[i]}", "value") for i in range(5)]
)
def actualizar_simulador_p3(n_clicks, universidad, *inputs):
    inputs = [x if x is not None else 0 for x in inputs]
    return figura3_p3(trainSimulador.copy(),predecirSimulador.copy(),universidad, inputs)

#Popup inicial
@app.callback(
    Output("welcome-modal", "is_open"),
    [Input("close-modal", "n_clicks")],
    [State("welcome-modal", "is_open")],
)
def control_modal(n_clicks, is_open):
    if n_clicks: 
        return not is_open
    return is_open
from dash import dcc, html, Input, Output, State, callback_context

#Inicializar el simulador a 0
@app.callback(
    [Output(f"input-{grupos_densidad[i]}", "value") for i in range(len(grupos_densidad))],
    [Input("submit-button", "n_clicks")],
    prevent_initial_call=False
)
def inicializar_valores(n_clicks):
    ctx = callback_context
    if not ctx.triggered: 
        return [0] * len(grupos_densidad)
    raise dash.exceptions.PreventUpdate  
    
if __name__ == '__main__':
    app.run_server(debug=False, host='0.0.0.0', port=8050)

ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_f1976a59-5bc4-4f8e-a4cc-cd4b4431d359 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020C460157F0>: Failed to establish a new connection: [WinError 10049] La dirección solicitada no es válida en este contexto'))